In [1]:
import pandas as pd
import numpy as np
import re
import string
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import json

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


from plotly.graph_objs import *

In [2]:
#filename = 'elasticsearch/elasticsearch'
#filename = 'spring-framework/spring-framework'
#filename = 'spring-boot/spring-boot'
filename = 'hadoop/hadoop'
#filename = 'guava/guava'
#filename = 'mockito/mockito'
#filename = 'slf4j/slf4j'
token_count = 200

In [3]:
df = pd.read_json(filename + '-tokens.txt')
df = df.transpose()
df.columns = ['rev', 'Commit', 'Diff']
print(len(df))
df = df[df['Diff'] < 5000]
print(len(df))

19231
14601


In [4]:
df['Diff'].sum() / len(df)

1020

In [5]:
df['Commit'].sum() / len(df)

15

In [6]:
df1 = df[['Commit']]
df2 = df[['Diff']]

In [16]:
plot([Box(x=df1['Commit'].tolist(), name='Commit', marker = dict(
        color = '#FF4136',
    ), boxmean=True)])

'file:///home/salman/CS5704/Project/Preprocess/temp-plot.html'

In [256]:
plot([Box(x=df2['Diff'].tolist(), name='Diff', marker = dict(
        color = '#FF851B',
    ), boxmean=True)])

'file:///home/salman/CS5704/Project/Preprocess/temp-plot.html'

In [11]:
mymsg = 'HADOOP-10368. InputStream is not closed in VersionInfo ctor.  Contributed by Tsuyoshi OZAWA'

In [34]:
def remwithre(text, there=re.compile(re.escape('HADOOP')+'.......')):
    return there.sub('', text)

remwithre(mymsg)

' InputStream is not closed in VersionInfo ctor.  Contributed by Tsuyoshi OZAWA'

In [39]:

re.compile(re.escape('Contributed')+'.*').sub('', re.compile(re.escape('HADOOP')+'.......').sub('', mymsg))
#re.sub(r'HADOOP.*?.', '.', mymsg)

' InputStream is not closed in VersionInfo ctor.  '

In [47]:
data = pd.read_csv(filename + '.txt', sep='\t', header=None)
data.columns = ["RevNo", "PrevRev", "Rev", "Message", "Generated"]

print('length before preprocess', str(len(data)))

#remove empty string
data = data[data['Generated'] != '']
data = data[data['Message'] != '']

#remove NaN values
data = data[pd.notnull(data['Generated'])]
data = data[pd.notnull(data['Message'])]

#Remove merge commit and 'discarded' generated message
data.loc[:,'Message'] = data['Message'].apply(lambda x: re.compile(re.escape('Merge')+'.*').sub('',x))
data.loc[:,'Generated'] = data['Generated'].apply(lambda x: re.compile(re.escape('Discarded')+'.*').sub('', x))

#specific to hadoop
#data.loc[:,'Generated'] = data['Generated'].apply(lambda x: re.sub(r"[^a-zA-Z0-9]+", ' ', x))
data.loc[:,'Message'] = data['Message'].apply(lambda x: re.compile(re.escape('Contributed')+'.*').sub('', re.compile(re.escape('HADOOP')+'.......').sub('', x)))
data.loc[:,'Message'] = data['Message'].apply(lambda x: re.compile(re.escape('HDFS')+'.......').sub('', x))
data.loc[:,'Message'] = data['Message'].apply(lambda x: re.compile(re.escape('YARN')+'.......').sub('', x))

#remove special characters
#data.loc[:,'Generated'] = data['Generated'].apply(lambda x: re.sub(r"[^a-zA-Z0-9]+", ' ', x))
#data.loc[:,'Message'] = data['Message'].apply(lambda x: re.sub(r"[^a-zA-Z0-9]+", ' ', x))

#remove digits
#data.loc[:,'Generated'] = data['Generated'].apply(lambda x: x.translate(None, string.digits))
#data.loc[:,'Message'] = data['Message'].apply(lambda x: x.translate(None, string.digits))

#split camel case words
#data.loc[:,'Message'] = data['Message'].apply(lambda x: " ".join(m.group(0) for m in re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', x)))
#data.loc[:,'Generated'] = data['Generated'].apply(lambda x: " ".join(m.group(0) for m in re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', x)))

#remove small commit messages and long generated messages
#data.loc[:,'Message'] = data['Message'].apply(lambda x: '' if len(x) < 15 else x)
#data.loc[:,'Generated'] = data['Generated'].apply(lambda x: '' if len(x) > 400 else x)

#again check empty and NaN values
data = data[pd.notnull(data['Generated'])]
data = data[data['Generated'] != '']
data = data[pd.notnull(data['Message'])]
data = data[data['Message'] != '']

#make two new columns 
#data.loc[:,'Ori'] = data['Message'].apply(lambda x: x.lower())
#data.loc[:,'Gen'] = data['Generated'].apply(lambda x: x.lower())

#make tokens of the newly generated columns and store in two addtional columns
data.loc[:,'OriTok'] = data['Message'].apply(lambda x: [x.strip() for x in x.split(" ")])
data.loc[:,'GenTok'] = data['Generated'].apply(lambda x: [x.strip() for x in x.split(" ")])

print('length after preprocess', str(len(data)))

('length before preprocess', '19235')
('length after preprocess', '11310')


In [49]:
len(data)

11310

In [50]:
indices = df[df['Diff'] <= token_count].rev.tolist()
data_clean = data[data['Rev'].isin(indices)]
len(data_clean)

1643

In [51]:
oriVocab = set()
genVocab = set()
for index, row in data.iterrows():
    oriVocab.update(row['OriTok'])
    genVocab.update(row['GenTok'])

print(len(oriVocab))
print(len(genVocab))

19136
47193


In [52]:
file1 = file(filename +'-vocab.ori', 'w')
file2 = file(filename +'-vocab.gen', 'w')

for item in list(oriVocab):
    if len(item) <= 2:
        continue
    file1.write(item)
    file1.write('\n')
    
for item in list(genVocab):
    if len(item) <= 2:
        continue
    file2.write(item)
    file2.write('\n')

file1.close()
file2.close()

In [53]:
data_clean = data_clean.sample(frac=1).reset_index(drop=True)
total = len(data_clean)
n = int(total * 0.8)
m = total - n
train_data = data_clean.head(n)
test_data = data_clean.tail(m)
dev_data = test_data.tail(len(test_data) / 2)
test_data = test_data.head(len(test_data) - len(dev_data))

In [54]:
print(len(data_clean))
print(len(train_data))
print(len(dev_data))
print(len(test_data))

1643
1314
164
165


In [55]:
train_data['Message'].to_csv(filename + '-train.ori', sep='\n', index=False, header=False)
train_data['Generated'].to_csv(filename + '-train.gen', sep='\n', index=False, header=False)

test_data['Message'].to_csv(filename + '-test.ori', sep='\n', index=False, header=False)
test_data['Generated'].to_csv(filename + '-test.gen', sep='\n', index=False, header=False)

dev_data['Message'].to_csv(filename + '-validation.ori', sep='\n', index=False, header=False)
dev_data['Generated'].to_csv(filename + '-validation.gen', sep='\n', index=False, header=False)

In [ ]:
#df = pd.DataFrame(np.random.rand(10, 1), columns=['A'])
color = dict(boxes='DarkGreen', whiskers='DarkOrange', medians='DarkBlue', caps='Gray')
df1.plot.box(vert=False, positions=[1], color=color, sym='r+')
plt.savefig(filename + '-diff-box1.png', dpi=300)

#df = pd.DataFrame(np.random.rand(10, 1), columns=['A'])
color = dict(boxes='DarkGreen', whiskers='DarkOrange', medians='DarkBlue', caps='Gray')
df2.plot.box(vert=False, positions=[1], color=color, sym='r+')
plt.savefig(filename + '-diff-box2.png', dpi=300)